# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [33]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import sqlite3
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import pickle

from sklearn import tree


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///data_project.db')
    df = pd.read_sql("SELECT * FROM data_project", engine)
    X = df['message']
    y = df.iloc[:,4:41]
    labels = y.columns
    return X, y, labels

### 2. Write a tokenization function to process your text data

In [23]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def tokenize(text):
    '''
    input: text
    
    return: cleaned text data
    '''
    # normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", str(text).lower().strip())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=1), n_jobs=-1))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [24]:
X, y,labels = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred = pipeline.predict(X_test)


In [21]:
print(classification_report(y_test, y_pred, target_names=labels))


                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5066
               request       0.86      0.48      0.62      1188
                 offer       0.00      0.00      0.00        22
           aid_related       0.76      0.70      0.73      2770
          medical_help       0.68      0.08      0.14       532
      medical_products       0.90      0.08      0.14       344
     search_and_rescue       0.52      0.06      0.10       190
              security       0.00      0.00      0.00       115
              military       0.62      0.08      0.14       198
           child_alone       0.00      0.00      0.00         0
                 water       0.93      0.40      0.56       414
                  food       0.86      0.59      0.70       743
               shelter       0.85      0.43      0.57       581
              clothing       0.91      0.08      0.15       119
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vec',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd61146f400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
               oob_score=False, random_state=1, ve

In [18]:
def build_model():
    pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=1), n_jobs=-1))
    ])

    
    parameters = {
        #'vec__max_df':(0.5,1),
        'vec__max_features':(500, 1000),
        #'clf__estimator__n_estimators':[100,200],
        'clf__estimator__min_samples_split': [2,4],
    }
         

    cv = GridSearchCV(pipeline, param_grid=parameters, refit=True)
    
    return cv

In [25]:
import signal

from contextlib import contextmanager

import requests


DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}


def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler


@contextmanager
def active_session(delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import active session

    with active_session():
        # do long-running work here
    """
    token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
    headers = {'Authorization': "STAR " + token}
    delay = max(delay, MIN_DELAY)
    interval = max(interval, MIN_INTERVAL)
    original_handler = signal.getsignal(signal.SIGALRM)
    try:
        signal.signal(signal.SIGALRM, _request_handler(headers))
        signal.setitimer(signal.ITIMER_REAL, delay, interval)
        yield
    finally:
        signal.signal(signal.SIGALRM, original_handler)
        signal.setitimer(signal.ITIMER_REAL, 0)


def keep_awake(iterable, delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import keep_awake

    for i in keep_awake(range(5)):
        # do iteration with lots of work here
    """
    with active_session(delay, interval): yield from iterable


In [8]:
with active_session():
    cv = build_model()
    cv.fit(X_train, y_train)

In [9]:
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vec__max_df': (0.5, 1), 'vec__max_features': (None, 5000, 10000), 'clf__estimator__n_estimators': [50, 100, 200], 'clf__estimator__min_samples_split': [2, 4, 6], 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
with active_session():
    def load_data():
        engine = create_engine('sqlite:///data_project.db')
        df = pd.read_sql("SELECT * FROM data_project", engine)
        X = df['message']
        y = df.iloc[:,4:41]
        labels = y.columns
        return X, y, labels


    def display_results(cv, y_test, y_pred):
        print(classification_report(y_test, y_pred, target_names=labels))


    def main():
        X, y,labels = load_data()
        X_train, X_test, y_train, y_test = train_test_split(X, y)

        model = build_model()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
        display_results(model, y_test, y_pred)


    main()

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      4989
               request       0.81      0.53      0.64      1067
                 offer       0.00      0.00      0.00        33
           aid_related       0.73      0.71      0.72      2643
          medical_help       0.55      0.17      0.26       495
      medical_products       0.79      0.20      0.32       305
     search_and_rescue       0.62      0.14      0.22       154
              security       0.00      0.00      0.00       104
              military       0.62      0.12      0.21       210
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.53      0.64       413
                  food       0.81      0.73      0.77       726
               shelter       0.82      0.52      0.64       591
              clothing       0.75      0.36      0.49        92
                 money       0.90      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# try using SMV instead of random forest

In [34]:
pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(tree.DecisionTreeClassifier()))
    ])

In [12]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...bility=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
           n_jobs=1))])>

In [35]:
def build_model():
    pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(tree.DecisionTreeClassifier()))
    ])

    
    parameters = {
        'vec__max_df':(0.5,1),
        'vec__max_features':(500, 1000)
    }
         

    cv = GridSearchCV(pipeline, param_grid=parameters, refit=True)
    
    return cv

In [36]:
with active_session():
    def load_data():
        engine = create_engine('sqlite:///data_project.db')
        df = pd.read_sql("SELECT * FROM data_project", engine)
        X = df['message']
        y = df.iloc[:,4:41]
        labels = y.columns
        return X, y, labels


    def display_results(cv, y_test, y_pred):
        print(classification_report(y_test, y_pred, target_names=labels))


    def main():
        X, y,labels = load_data()
        X_train, X_test, y_train, y_test = train_test_split(X, y)

        model = build_model()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
        display_results(model, y_test, y_pred)


    main()

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4975
               request       0.33      0.00      0.00      1094
                 offer       0.00      0.00      0.00        28
           aid_related       0.66      0.01      0.01      2729
          medical_help       0.20      0.00      0.01       509
      medical_products       0.40      0.01      0.01       311
     search_and_rescue       0.00      0.00      0.00       179
              security       0.00      0.00      0.00       118
              military       1.00      0.00      0.01       227
           child_alone       0.00      0.00      0.00         0
                 water       0.33      0.00      0.01       448
                  food       0.38      0.00      0.01       735
               shelter       0.30      0.00      0.01       626
              clothing       0.00      0.00      0.00       105
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [19]:
final_model = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize,max_features = 1000 )),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200, random_state=1, min_samples_split= 2), n_jobs=-1))
    ])


In [20]:
filename = 'finalized_model.sav'
pickle.dump(final_model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.